# Importing Important Libraries

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
%matplotlib inline

# Dataset Import and Preparations

## Importing the dataset

In [ ]:
dataset = pd.read_csv("../input/restaurantrecommendationdata/train_100k.csv")
dataset.head()

## Checking whether any columns has missing value

In [ ]:
dataset.isnull().sum()

## Let's check the number of columns or rows for our restaurant dataset

In [ ]:
dataset.shape

### So our restaurant dataset has a total of 17 columns and 100000 rows. However, the customer id is of object type and our model is not able to handle object type data. So, we need to do something about it. In the next section we will preprocess our data and generate unique id number for customers. Since, there is no missing value we don't need to do any imputation for this dataset

# Collaborative Filtering Recommendation

### In this section we will build an interaction matrix that will be used to train our collaborative filetering recommendation. As, collaborative filtering works with the interaction between item and user we can't use the dataset as it is

## Important Functions

In [ ]:
def generate_id(df, column_name, id_column):
    
    value_list = df[column_name].unique().tolist()
    int_value_list = [i for i in range(len(value_list))]
    id_list = []
    
    for i in df[column_name]:
        id_list.append(int_value_list[ value_list.index(i) ])
    
    df[id_column] = id_list

def similar_user_recs(user, interaction_mat, user_sim):

    if user not in interaction_mat.columns:
        return('No data available on user {}'.format(user))

    sim_users = user_sim.sort_values(by=user, ascending=False).index[1:11]
    best = []
    most_common = {}

    for i in sim_users:
        max_score = interaction_mat.loc[:, i].max()
        best.append(interaction_mat[interaction_mat.loc[:, i]==max_score].index.tolist())
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_list[:5]   

## Unique id generation for customer_id column

In [ ]:
generate_id(dataset, 'customer_id', 'id_customer')
dataset.head()

### Let's see the unique restaurant id's that is present in our dataset. We will need this later in our process.

In [ ]:
restaurant_id_list = list(dataset['restaurant_id'].unique())
restaurant_id_list[:10]

### Also the unique customer id that we generated will also be used later. I put it here just to have a glimpse of what we are expecting

In [ ]:
customer_list = list(dataset['id_customer'].unique())
customer_list[:10]

## Interaction Matrix Creation

### In this section we will generate an interaction matrix that will be used for our recommendation system. First we will generate a dictionary then we will parse that dictionary to our pandas dataset.

In [ ]:
interaction_dict = dict()
for id_name in restaurant_id_list:
    temp_user_order = []
    for customer in customer_list:
        temp_user_order.append( dataset[ (dataset['id_customer']==customer) & (dataset['restaurant_id']==id_name) ].shape[0] )
    interaction_dict[str(id_name)] = temp_user_order
#interaction_dict

In [ ]:
interaction_dataset = pd.DataFrame(interaction_dict)
interaction_dataset

### Now, we have a dataset that has number of total restaurant as columns and number of total customer id's as rows. This represents our interaction matrix where each row represents how many the user has made order from that particular restaurant. This will be used to make weight for user/item interaction in out interaction matrix.

## Transpose Dataset to fit the interaction matrix

In [ ]:
interaction_dataset = interaction_dataset.T
interaction_dataset

## Here we will convert our dataset to sparse array as it will be helpful for our system to work on interaction matrix efficiently

In [ ]:
inter_sparse_data = sp.sparse.csr_matrix(interaction_dataset.values)
inter_sparse_data

## Finding Cosine Similarity Between User/Item

In [ ]:
user_similarity = cosine_similarity(inter_sparse_data.T)
user_sim = pd.DataFrame(user_similarity, index = interaction_dataset.columns, columns = interaction_dataset.columns)
user_sim

# Making Recommendations

In [ ]:
similar_user_recs(0, interaction_dataset, user_sim)

### Here the first value the tuple represents the restaurant id and the second value represents the user that is most similar in terms of interactions.

 # Conclusion
 ### This is a test on the recommendation system on the restaurant dataset. It can be enhanced through applying various conditions and improving weight on the interaction matrix. The location also has a huge impact on the interaction system. However, for simplified learning I have ignored it for now. Kindly comment if you have any suggestions or advice regarding this notebook